# Task 1 - Train on the NEU train.

In [1]:
import os
import numpy as np

In [2]:
train_crazing_dir = os.path.join('NEU-DET/train/images/crazing')
train_inclusion_dir = os.path.join('NEU-DET/train/images/inclusion')
train_patches_dir = os.path.join('NEU-DET/train/images/patches')
train_pitted_surface_dir = os.path.join('NEU-DET/train/images/pitted_surface')
train_rolledin_scale_dir = os.path.join('NEU-DET/train/images/rolled-in_scale')
train_scratches_dir = os.path.join('NEU-DET/train/images/scratches')

In [3]:
train_crazing_names = os.listdir(train_crazing_dir)
train_inclusion_names = os.listdir(train_inclusion_dir)
train_patches_names = os.listdir(train_patches_dir)
train_pitted_surface_names = os.listdir(train_pitted_surface_dir)
train_rolledin_scale_names = os.listdir(train_rolledin_scale_dir)
train_scratches_names = os.listdir(train_scratches_dir)

In [4]:
print('total training crazing images:', len(train_crazing_names))
print('total training inclusion images:', len(train_inclusion_names))
print('total training patches images:', len(train_patches_names))
print('total training pitted_surface images:', len(train_pitted_surface_names))
print('total training rolled_in_scale images:', len(train_rolledin_scale_names))
print('total training scratches images:', len(train_scratches_names))

total training crazing images: 240
total training inclusion images: 240
total training patches images: 240
total training pitted_surface images: 240
total training rolled_in_scale images: 240
total training scratches images: 240


In [5]:
import tensorflow as tf


In [6]:
model = tf.keras.models.Sequential([
                                    # First Convolution
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # Second Convolution
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # Third Convolution
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(6, activation='softmax'),])




In [7]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 32)       0

In [8]:
from tensorflow.keras.optimizers import RMSprop, Adam

model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# Rescaling images
data_generator = ImageDataGenerator(rescale=1/255)

train_generator = data_generator.flow_from_directory(
    directory = 'NEU-DET/train/images/',
    target_size = (224, 224),
    batch_size = 60,
    class_mode = 'categorical',
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    directory = 'NEU-DET/validation/images/',
    target_size = (224, 224),
    batch_size = 60,
    class_mode = 'categorical',
    subset='training'

)

Found 1440 images belonging to 6 classes.
Found 360 images belonging to 6 classes.


In [10]:
# Load the TensorBoard notebook extension
import datetime

%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(train_generator,steps_per_epoch=8,epochs=5, verbose=1,callbacks=[tensorboard_callback])
%tensorboard --logdir logs/fit

Epoch 1/5
8/8 [==============================] - 20s 3s/step - loss: 0.5222 - accuracy: 0.1500
Epoch 2/5
8/8 [==============================] - 20s 2s/step - loss: 0.4704 - accuracy: 0.1667
Epoch 3/5
8/8 [==============================] - 20s 3s/step - loss: 0.4423 - accuracy: 0.2417
Epoch 4/5
8/8 [==============================] - 19s 2s/step - loss: 0.4170 - accuracy: 0.2958
Epoch 5/5
8/8 [==============================] - 20s 3s/step - loss: 0.3532 - accuracy: 0.4729


Reusing TensorBoard on port 6006 (pid 12072), started 3:21:30 ago. (Use '!kill 12072' to kill it.)

In [11]:
history.history

{'loss': [0.5222132205963135,
  0.4704251289367676,
  0.4422782063484192,
  0.41697895526885986,
  0.35323062539100647],
 'accuracy': [0.15000000596046448,
  0.1666666716337204,
  0.24166665971279144,
  0.2958333194255829,
  0.4729166626930237]}

## Test on NEU test, and the whole ENEU.

In [12]:
tf.config.run_functions_eagerly(True)
model.evaluate(validation_generator)

C:\Users\91800\anaconda3\envs\MTP\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


6/6 [==============================] - 5s 804ms/step - loss: 0.3326 - accuracy: 0.4528


[0.3325871229171753, 0.45277777314186096]

In [32]:
validation_generator_ENEU = data_generator.flow_from_directory(
    directory = 'ENEU/',
    target_size = (224, 224),
    batch_size = 60,
    class_mode = 'categorical'
)

Found 10800 images belonging to 6 classes.


In [33]:
model.evaluate(validation_generator_ENEU)

180/180 [==============================] - 144s 799ms/step - loss: 0.5535 - accuracy: 0.1527


[0.5534840226173401, 0.15268518030643463]